<a href="https://colab.research.google.com/github/thasliya0404/class-activity/blob/main/data_ingestion_techniques_Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

data ingestion

is the process of gathering,meaning,and utilizing data eficiently

steps

1. collection:gather raw data from various sources


2. transformation:clean, normalize, enrich the data


3.data loading:move the transformed data in to the target system


types

1.batch

2.real time

3.microbatching

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/velicki/Weather_Data_Analysis_Project/main/Weather_Data.csv'
data = pd.read_csv(url)
data

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog
...,...,...,...,...,...,...,...,...
8779,12/31/2012 19:00,0.1,-2.7,81,30,9.7,100.13,Snow
8780,12/31/2012 20:00,0.2,-2.4,83,24,9.7,100.03,Snow
8781,12/31/2012 21:00,-0.5,-1.5,93,28,4.8,99.95,Snow
8782,12/31/2012 22:00,-0.2,-1.8,89,28,9.7,99.91,Snow


In [ ]:
data.columns

Index(['Date/Time', 'Temp_C', 'Dew Point Temp_C', 'Rel Hum_%',
       'Wind Speed_km/h', 'Visibility_km', 'Press_kPa', 'Weather'],
      dtype='object')

In [ ]:
import time
from IPython.display import Audio, display

In [ ]:
data.columns = data.columns.str.strip().str.replace(' ','_').str.replace('/','_')
data.columns

Index(['Date_Time', 'Temp_C', 'Dew_Point_Temp_C', 'Rel_Hum_%',
       'Wind_Speed_km_h', 'Visibility_km', 'Press_kPa', 'Weather'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date_Time         8784 non-null   object 
 1   Temp_C            8784 non-null   float64
 2   Dew_Point_Temp_C  8784 non-null   float64
 3   Rel_Hum_%         8784 non-null   int64  
 4   Wind_Speed_km_h   8784 non-null   int64  
 5   Visibility_km     8784 non-null   float64
 6   Press_kPa         8784 non-null   float64
 7   Weather           8784 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 549.1+ KB


In [ ]:
data['Date_Time'] = pd.to_datetime(data['Date_Time'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date_Time         8784 non-null   datetime64[ns]
 1   Temp_C            8784 non-null   float64       
 2   Dew_Point_Temp_C  8784 non-null   float64       
 3   Rel_Hum_%         8784 non-null   int64         
 4   Wind_Speed_km_h   8784 non-null   int64         
 5   Visibility_km     8784 non-null   float64       
 6   Press_kPa         8784 non-null   float64       
 7   Weather           8784 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 549.1+ KB


In [ ]:
data.isna().sum()

,0
Date_Time,0
Temp_C,0
Dew_Point_Temp_C,0
Rel_Hum_%,0
Wind_Speed_km_h,0
Visibility_km,0
Press_kPa,0
Weather,0


In [ ]:
data.head()

,Date_Time,Temp_C,Dew_Point_Temp_C,Rel_Hum_%,Wind_Speed_km_h,Visibility_km,Press_kPa,Weather
0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [ ]:
def alert_high_temp(row):
  if (row['Temp_C'] > 3):
    print(f"Alert!!! Temperature is too high: {row['Temp_C']} Celcius")
    display(Audio('/content/siren-alert-96052.mp3',autoplay=True))
    return True
  else:
    print(f"Tempertaure is normal: {row['Temp_C']} Celcius")
    return False


def stream_with_alert(data,delay=1):
  for _, row in data.iterrows():
    print(f"{row['Date_Time']} - Temperature : {row['Temp_C']} Celsius")
    if alert_high_temp(row):
      print("High temperature alert triggered. Stopping Stream processing")
      break
    time.sleep(delay)

In [ ]:
stream_with_alert(data)

2012-01-01 00:00:00 - Temperature : -1.8 Celsius
Tempertaure is normal: -1.8 Celcius
2012-01-01 01:00:00 - Temperature : -1.8 Celsius
Tempertaure is normal: -1.8 Celcius
2012-01-01 02:00:00 - Temperature : -1.8 Celsius
Tempertaure is normal: -1.8 Celcius
2012-01-01 03:00:00 - Temperature : -1.5 Celsius
Tempertaure is normal: -1.5 Celcius
2012-01-01 04:00:00 - Temperature : -1.5 Celsius
Tempertaure is normal: -1.5 Celcius
2012-01-01 05:00:00 - Temperature : -1.4 Celsius
Tempertaure is normal: -1.4 Celcius
2012-01-01 06:00:00 - Temperature : -1.5 Celsius
Tempertaure is normal: -1.5 Celcius
2012-01-01 07:00:00 - Temperature : -1.4 Celsius
Tempertaure is normal: -1.4 Celcius
2012-01-01 08:00:00 - Temperature : -1.4 Celsius
Tempertaure is normal: -1.4 Celcius
2012-01-01 09:00:00 - Temperature : -1.3 Celsius
Tempertaure is normal: -1.3 Celcius
2012-01-01 10:00:00 - Temperature : -1.0 Celsius
Tempertaure is normal: -1.0 Celcius
2012-01-01 11:00:00 - Temperature : -0.5 Celsius
Tempertaure is n

High temperature alert triggered. Stopping Stream processing


# ***kafka***

Brokers, Producers, Consumers, and Zookeeper - Scaler Topics. In Kafka, a producer sends messages to a topic, which is divided into partitions for parallel processing.

These partitions are stored by brokers within a Kafka cluster, which are coordinated by ZooKeeper.

Consumers subscribe to topics and read messages from partitions, tracking their progress using an offset.

producer

topic

partition

consumer/group

broker

zookeeper

offset

kafka cluster

**Producers**

Send data (messages) to Kafka topics.
Can optimize message delivery by balancing, serializing, and compressing messages across partitions.

**Topics**

Logical categories for storing messages.
Messages within a topic are organized into partitions.

**Partitions**

Subdivisions of a topic that hold messages in an ordered, immutable sequence.
Enable parallel processing by allowing multiple consumers to read different partitions simultaneously.

**Brokers**

Kafka servers that store data, handle read/write requests, and manage partitions.
They form a Kafka cluster and distribute the data load across the cluster.

**Kafka** **Cluster**

A collection of one or more brokers working together to provide high availability, scalability, and fault tolerance.

**ZooKeeper** (or KRaft)

Used for cluster coordination and managing metadata.
Tracks the status of brokers, performs leader election for partitions, and notifies producers and consumers of cluster changes.

**Consumers**

Subscribe to topics to read messages.
Pull messages from partitions rather than having them pushed by Kafka.

**Consumer Group**

A group of consumers that share the responsibility of consuming messages from a topic.
Each partition in a topic is consumed by only one consumer within a given group, ensuring each message is processed only once by the group.

**Offset**

A unique, sequential identifier for each message within a partition.
Consumers maintain a record of the last consumed offset for each partition, allowing them to resume reading from where they left off after a crash or restart.